In [1]:
#THIS IS FOR USING JUST ONE GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import json
from tqdm import tqdm

/home/maria/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load dataset

In [4]:
with open("perfect_subset_phi.json", "r") as f:
    dataset = json.load(f)

print(f"Loaded {len(dataset)} examples.")

Loaded 201 examples.


-Load finetune model from Hugging Face

In [5]:

model = AutoModelForCausalLM.from_pretrained("karou1182001/fine-tuned-phi")
tokenizer = AutoTokenizer.from_pretrained("karou1182001/fine-tuned-phi")
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")
print("Model loaded from Hugging Face.")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 33.61it/s]


Model loaded from Hugging Face.


Evaluate dataset

In [ ]:
correct = 0
first_printed = False

for item in tqdm(dataset, desc="Evaluating..."):
    question = item["question"].strip()
    gold = str(item["gold_num"])

    # Zero-shot prompt 
    prompt = question + "\nA:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    pred_numbers = re.findall(r"\d+", decoded)
    prediction = pred_numbers[-1] if pred_numbers else None

    if not first_printed:
        print("\nFirst Example:")
        print(f"Question: {question}")
        print(f"Expected (gold): {gold}")
        print(f"Predicted: {prediction}")
        first_printed = True

    if prediction == gold:
        correct += 1

total = len(dataset)
accuracy = correct / total * 100

print(f"\nCorrect predictions: {correct}/{total}")
print(f"Accuracy on dataset (zero-shot): {accuracy:.2f}%")
